In [ ]:
!pip install sktime
!pip install pmdarima

In [ ]:
from pmdarima import auto_arima
import pandas as pd
import polars as pl
import numpy as np
import logging
import time
from os import listdir
from os.path import isfile, join
import datetime
from google.colab import drive
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.preprocessing import StandardScaler
from math import sqrt
from prophet import Prophet
import matplotlib.pyplot as plt
from statsmodels.tsa.vector_ar.var_model import VAR
from statsmodels.tsa.ar_model import AutoReg
drive.mount('/content/drive')
path_default = '/content/drive/MyDrive/TCC-MBA_USP_ESALQ/'

In [ ]:
############## Funções de tratativa dos dados ##############

####### Função que trata os dados e insere estação, região e UF no dataframe meteorológico #######
def insert_uf_df(df_info, df_general):
    estacao = df_info.row(2)[1]
    regiao = df_info.row(0)[1]
    uf = df_info.row(1)[1]
    indice_max = len(df_general)
    lista_estacao = [estacao for j in range(indice_max)]
    lista_regioes = [regiao for j in range(indice_max)]
    lista_uf = [uf for j in range(indice_max)]
    series_estacao = pl.Series("ESTAÇÃO", lista_estacao)
    series_regioes = pl.Series("REGIÕES", lista_regioes)
    series_uf = pl.Series("UF", lista_uf)
    df_general.insert_column(0, series_estacao)
    df_general.insert_column(1, series_regioes)
    df_general.insert_column(2, series_uf)
    return df_info, df_general


####### Função de leitura dos arquivos CSV de produção #######
def read_all_csv_from_path(path):
    dfs = []
    onlyfiles = [f for f in listdir(path) if isfile(join(path, f))]
    for filename in onlyfiles:
        df = pl.read_csv(join(path, filename), separator=";")
        dfs.append(df)
    return pl.concat(dfs)


####### Função de leitura dos arquivos CSV meteorológicos #######
def read_all_met_csv_from_path(path):
    dfs_info = []
    dfs_general = []
    onlyfiles = [f for f in listdir(path) if isfile(join(path, f))]
    for filename in onlyfiles:
        df_general = pl.read_csv(join(path, filename), has_header=False, skip_rows=8, encoding='latin1', separator=";")
        df_info = pl.read_csv(join(path, filename), has_header=False, encoding='latin1', n_rows=8, separator=";", truncate_ragged_lines=True)
        df_general = take_header_weather(df_general)
        df_info, df_general = insert_uf_df(df_info, df_general)
        dfs_info.append(df_info)
        dfs_general.append(df_general)
    return pl.concat(dfs_info), pl.concat(dfs_general)

####### Função que pega o cabeçalho do arquivo meteorológico #######
def take_header_weather(df):
    lista = list(df.row(0))
    lista[-1] = 'None Column'
    df.columns = lista
    df = df.slice(1, None)
    return df

####### Função que transforma data e hora para um datetime #######
def data_and_hour_to_datetime(df):
  date = df['DATA (YYYY-MM-DD)'].to_list()
  hour = df['HORA (UTC)'].to_list()
  datetime_list = [datetime.datetime.strptime(date[i] + " " + hour[i], "%Y-%m-%d %H:%M") for i in range(len(date))]
  data_series = pl.Series('DATETIME', datetime_list)
  df = df.insert_column(-1, data_series)
  df = df.drop('DATA (YYYY-MM-DD)')
  df = df.drop('HORA (UTC)')
  return df


######## Função que pega as médias de uma lista de valores #######
def avg_function(lista):
    if not lista:
        return 0
    return sum(lista) / len([i for i in lista if i != 0])


####### Função que pega as médias dos dados meteorológicos para cada UF e Ano #######
def take_mean_weather(df):
    ano_list = [i.year for i in df['DATETIME'].to_list()]
    data_series = pl.Series('ANO', ano_list)
    df = df.insert_column(-1, data_series)
    lista_ano = list(set(df["ANO"].to_list()))
    uf_lista = list(set(df["UF"].to_list()))
    dfs_list = []

    for ano in lista_ano:
      for ufs in uf_lista:
        df_tot = df.filter((pl.col("UF") == ufs))
        list_df = []
        lists_all = []

        lists_all.append(list(map(float, map(lambda x: x.replace(',', '.'),df_tot['PRECIPITAÇÃO TOTAL, HORÁRIO (mm)'].to_list()))))
        lists_all.append(list(map(float, map(lambda x: x.replace(',', '.'),df_tot['TEMPERATURA MÁXIMA NA HORA ANT. (AUT) (°C)'].to_list()))))
        lists_all.append(list(map(float, map(lambda x: x.replace(',', '.'),df_tot['TEMPERATURA MÍNIMA NA HORA ANT. (AUT) (°C)'].to_list()))))
        lists_all.append(list(map(float, map(lambda x: x.replace(',', '.'),df_tot['UMIDADE RELATIVA DO AR, HORARIA (%)'].to_list()))))
        lists_all.append(list(map(float, map(lambda x: x.replace(',', '.'),df_tot['VENTO, VELOCIDADE HORARIA (m/s)'].to_list()))))
        lists_all.append(list(map(float, map(lambda x: x.replace(',', '.'),df_tot['PRESSAO ATMOSFERICA AO NIVEL DA ESTACAO, HORARIA (mB)'].to_list()))))

        lists_all[0] = [elemento for elemento in lists_all[0] if elemento > 0]
        lists_all[1] = [elemento for elemento in lists_all[1] if elemento > 0]
        lists_all[2] = [elemento for elemento in lists_all[2] if elemento > 0]
        lists_all[3] = [elemento for elemento in lists_all[3] if elemento > 0]
        lists_all[4] = [elemento for elemento in lists_all[4] if elemento > 0]
        lists_all[5] = [elemento for elemento in lists_all[5] if elemento > 0]

        list_df.append(avg_function(lists_all[0]))
        list_df.append(avg_function(lists_all[1]))
        list_df.append(avg_function(lists_all[2]))
        list_df.append(avg_function(lists_all[3]))
        list_df.append(avg_function(lists_all[4]))
        list_df.append(avg_function(lists_all[5]))

        dict_list = {'PRECIPITAÇÃO TOTAL, HORÁRIO (mm)': list_df[0], 'TEMPERATURA MÁXIMA, HORÁRIO (°C)':list_df[1], 'TEMPERATURA MÍNIMA, HORÁRIO (°C)': list_df[2],
                  'UMIDADE RELATIVA, HORÁRIO (%)': list_df[3], 'VELOCIDADE DO VENTO, HORÁRIO (m/s)': list_df[4], 'PRESSÃO ATMOSFERICA, HORÁRIO (mB)': list_df[5],
                  'UF': ufs, 'Ano': ano}

        dfs_list.append(pl.DataFrame(dict_list))

    return pl.concat(dfs_list)


####### Função que pega as médias dos Valores de Medidas para cada UF e Ano #######
def take_avg_df(df):
  lista_ano = list(set(df['Ano'].to_list()))
  lista_uf = list(set(df['UF'].to_list()))
  lista_medidas = list(set(df['Nomes de medida'].to_list()))
  variavel = []
  quartel = []
  ano = []
  nivel = []
  territorio = []
  produto = []
  valores_medida = []
  ufs_list = []
  medidas_list = []
  ano_list = []

  for ano in lista_ano:
    for ufs in lista_uf:
      for medidas in lista_medidas:
        df_tot = df.filter((pl.col("UF") == ufs) & (pl.col("Nomes de medida") == medidas) & (pl.col("Ano") == ano))

        if df_tot.is_empty():
          break

        list_valores = [float(i.replace(',', '.')) for i in df_tot['Valores de medida'].to_list()]

        medidas_list.append(medidas)
        ufs_list.append(ufs)
        variavel.append(df_tot['Variável'].to_list()[0])
        quartel.append(df_tot['Quartel'].to_list()[0])
        ano_list.append(ano)
        nivel.append(df_tot['Nível'].to_list()[0])
        territorio.append(df_tot['Território'].to_list()[0])
        produto.append(df_tot['Produto'].to_list()[0])
        valores_medida.append(avg_function(list_valores))

  dict_list = {'UF': ufs_list, 'Variável': variavel, 'Quartel': quartel, 'Ano': ano_list, 'Nível': nivel, 'Território': territorio,
               'Produto': produto, 'Nomes de medida':medidas_list, 'Valores de medida': valores_medida}

  return pl.DataFrame(dict_list)


####### Função que transforma o ano em datetime.date #######
def year_to_date(df):
  ano_list = [datetime.date(i, 1, 1) for i in df['Ano'].to_list()]
  df = df.drop('Ano')
  data_series = pl.Series('Ano', ano_list)
  df = df.insert_column(-1, data_series)
  return df



####### Função que faz a previsão via Prophet #######
def prophet_forecast(df, ylabel, save_path):
    df_pandas = df.to_pandas()
    df_pandas = df_pandas.rename(columns={'Ano': 'ds', 'Valores de medida': 'y'})
    uf_lista = list(set(df_pandas["UF"].to_list()))
    fig, axs = plt.subplots(9, 3, figsize=(15, 30))
    content_path = "/content/drive/MyDrive/TCC-MBA_USP_ESALQ/"
    index = 0

    mae_list = []
    mse_list = []
    rmse_list = []
    mape_list = []
    r2_list = []
    uf_valid = []

    for ufs in uf_lista:
        i, j = divmod(index, 3)
        df_uf = df_pandas[df_pandas['UF'] == ufs]

        if df_uf.dropna().shape[0] < 2:
            print(f'Pular UF {ufs}: menos de 2 linhas não-NaN')
            continue

        model = Prophet()
        model.fit(df_uf)

        future = model.make_future_dataframe(periods=3, freq='Y')
        forecast = model.predict(future)
        forecast_within = forecast.loc[forecast['ds'].isin(df_uf['ds'])]['yhat'].values

        y_true = df_uf['y'].values
        mae = mean_absolute_error(y_true, forecast_within)
        mse = mean_squared_error(y_true, forecast_within)
        rmse = np.sqrt(mse)
        mape = np.mean(np.abs((y_true - forecast_within) / y_true)) * 100
        r2 = r2_score(y_true, forecast_within)

        mae_list.append(mae)
        mse_list.append(mse)
        rmse_list.append(rmse)
        mape_list.append(mape)
        r2_list.append(r2)
        uf_valid.append(ufs)

        model.plot(forecast, ax=axs[i, j])
        axs[i, j].set_title(f'UF: {ufs}')
        axs[i, j].set_xlabel('Ano')
        axs[i, j].set_ylabel(ylabel)

        index += 1

    plt.tight_layout()
    plt.savefig(content_path + save_path)
    plt.show()

    metrics_df = pd.DataFrame({
        'UF': uf_valid,
        'MAE': mae_list,
        'MSE': mse_list,
        'RMSE': rmse_list,
        'MAPE': mape_list,
        'R2': r2_list,
        'Method': ['Prophet' for i in uf_valid]
    })

    return metrics_df


####### Função que faz a previsão via ARIMA #######
def autoarima_forecast(df, ylabel, save_path):
    df_pandas = df.to_pandas()
    df_pandas = df_pandas.rename(columns={'Ano': 'ds', 'Valores de medida': 'y'})
    uf_lista = list(set(df_pandas["UF"].to_list()))
    content_path = "/content/drive/MyDrive/TCC-MBA_USP_ESALQ/"

    fig, axs = plt.subplots(9, 3, figsize=(15, 30))
    index = 0
    periods = 2

    mae_list = []
    mse_list = []
    rmse_list = []
    mape_list = []
    r2_list = []
    uf_valid = []

    for ufs in uf_lista:
        i, j = divmod(index, 3)

        df_uf = df_pandas[df_pandas['UF'] == ufs]
        df_uf = df_uf.sort_values(by=['ds'])

        if df_uf.dropna().shape[0] < 2:
            print(f'Pular UF {ufs}: menos de 2 linhas não-NaN')
            continue

        model = auto_arima(df_uf['y'], seasonal=True, m=1)
        forecast_within = model.predict_in_sample()
        forecast = model.predict(n_periods=periods)

        mae = mean_absolute_error(df_uf['y'], forecast_within)
        mse = mean_squared_error(df_uf['y'], forecast_within)
        rmse = np.sqrt(mse)
        mape = np.mean(np.abs((df_uf['y'] - forecast_within) / df_uf['y'])) * 100
        r2 = r2_score(df_uf['y'], forecast_within)

        mae_list.append(mae)
        mse_list.append(mse)
        rmse_list.append(rmse)
        mape_list.append(mape)
        r2_list.append(r2)
        uf_valid.append(ufs)

        future_dates = [datetime.date(i, 1, 1) for i in list(range(df_uf['ds'].max().year + 1, df_uf['ds'].max().year + (1+periods)))]
        forecast_df = pd.DataFrame({
            'ds': future_dates,
            'y': forecast
        })
        df_uf_filtered = df_uf[['ds', 'y']]
        tot = pd.concat([df_uf_filtered, forecast_df])

        axs[i, j].scatter(tot['ds'], tot['y'], label='Observado', color='black', s=10)
        axs[i, j].plot(df_uf['ds'], forecast_within, label='ARIMA Regressão', color='tab:blue')
        axs[i, j].set_title(f'UF: {ufs}')
        axs[i, j].set_xlabel('Ano')
        axs[i, j].set_ylabel(ylabel)
        axs[i, j].legend()

        index += 1

    plt.tight_layout()
    plt.savefig(content_path + save_path)
    plt.show()

    metrics_df = pd.DataFrame({
        'UF': uf_valid,
        'MAE': mae_list,
        'MSE': mse_list,
        'RMSE': rmse_list,
        'MAPE': mape_list,
        'R2': r2_list,
        'Method': ['Arima' for i in uf_valid]
    })

    return metrics_df


####### Função que faz a previsão via VAR #######
def var_forecast(data, ylabel, save_path):
    if isinstance(data, pl.DataFrame):
        data = data.to_pandas()
    uf_lista = list(set(data["UF"].to_list()))
    content_path = "/content/drive/MyDrive/TCC-MBA_USP_ESALQ/"
    periods=2

    fig, axs = plt.subplots(9, 3, figsize=(15, 30))
    index = 0

    mae_list = []
    mse_list = []
    rmse_list = []
    mape_list = []
    r2_list = []
    uf_valid = []

    for ufs in uf_lista:
        i, j = divmod(index, 3)

        df_uf = data[data['UF'] == ufs]
        df_uf = df_uf.sort_values(by=['Ano'])
        df_uf = df_uf.drop(columns=['Variável', 'Quartel', 'Nível', 'Território', 'Produto', 'Nomes de medida', 'UF'])

        lista_anos = df_uf['Ano']
        df_aux = df_uf.copy()
        df_uf.set_index('Ano', inplace=True)
        df_uf.index = pd.to_datetime(df_uf.index, format='%Y')
        df_uf.columns = df_uf.columns.str.strip()
        df_uf = df_uf.apply(pd.to_numeric, errors='coerce')
        lag_order = 1
        n_ahead = 2
        df_uf = df_uf.dropna(subset=['Valores de medida'])

        if len(df_uf) <= 4:
          continue

        print(df_uf)

        if df_uf.isnull().values.any() or np.isinf(df_uf.values).any():
          time_series = df_uf['Valores de medida']
          ar_model = AutoReg(time_series, lags=lag_order).fit()
          forecast = ar_model.predict(start=len(time_series), end=len(time_series) + n_ahead - 1)
          forecast_fitted_df = pd.DataFrame(forecast,
                                            index=pd.date_range(start=df_uf.index[-1], periods=len(forecast) + 1, freq='Y')[1:],
                                            columns=['Valores de medida'])
          forecast_within = ar_model.fittedvalues
          actual_values = df_uf['Valores de medida'].iloc[lag_order:]
          actual_dates = df_aux['Ano'].iloc[lag_order:]

          mae = mean_absolute_error(actual_values, forecast_within)
          mse = mean_squared_error(actual_values, forecast_within)
          rmse = np.sqrt(mse)
          mape = np.mean(np.abs((actual_values - forecast_within) / actual_values)) * 100
          r2 = r2_score(actual_values, forecast_within)

          mae_list.append(mae)
          mse_list.append(mse)
          rmse_list.append(rmse)
          mape_list.append(mape)
          r2_list.append(r2)
          uf_valid.append(ufs)

          future_dates = [datetime.date(i, 1, 1) for i in list(range(lista_anos.max().year + 1, lista_anos.max().year + (1+periods)))]
          forecast_df = pd.DataFrame({
              'Ano': future_dates,
              'Valores de medida': forecast_fitted_df['Valores de medida'].tolist()
              })
          df_uf_filtered = df_aux[['Ano', 'Valores de medida']]
          tot = pd.concat([df_uf_filtered, forecast_df])

          axs[i, j].scatter(tot['Ano'], tot['Valores de medida'], label='Observado', color='black', s=10)
          axs[i, j].plot(forecast_within.index, forecast_within, label='Previsão AR', color='tab:blue')

          axs[i, j].set_title(f'UF: {ufs}')
          axs[i, j].set_xlabel('Ano')
          axs[i, j].set_ylabel(ylabel)
          axs[i, j].legend()

          index += 1

        else:
          lag_order = 1
          n_ahead = 2
          model = VAR(df_uf)
          model_fitted = model.fit(lag_order)
          forecast = model_fitted.forecast(df_uf.values[-lag_order:], steps=n_ahead)
          forecast_fitted_df = pd.DataFrame(forecast,
                            index=pd.date_range(start=df_uf.index[-1], periods=len(forecast) + 1, freq='Y')[1:],
                            columns=df_uf.columns)
          forecast_within = model_fitted.fittedvalues['Valores de medida']
          actual_values = df_uf['Valores de medida'].iloc[lag_order:]
          actual_dates = df_aux['Ano'].iloc[lag_order:]

          mae = mean_absolute_error(actual_values, forecast_within)
          mse = mean_squared_error(actual_values, forecast_within)
          rmse = np.sqrt(mse)
          mape = np.mean(np.abs((actual_values - forecast_within) / actual_values)) * 100
          r2 = r2_score(actual_values, forecast_within)

          mae_list.append(mae)
          mse_list.append(mse)
          rmse_list.append(rmse)
          mape_list.append(mape)
          r2_list.append(r2)
          uf_valid.append(ufs)

          future_dates = [datetime.date(i, 1, 1) for i in list(range(lista_anos.max().year + 1, lista_anos.max().year + (1+periods)))]
          forecast_df = pd.DataFrame({
              'Ano': future_dates,
              'Valores de medida': forecast_fitted_df['Valores de medida'].tolist()
              })
          df_uf_filtered = df_aux[['Ano', 'Valores de medida']]
          tot = pd.concat([df_uf_filtered, forecast_df])

          axs[i, j].scatter(tot['Ano'], tot['Valores de medida'], label='Observado', color='black', s=10)
          axs[i, j].plot(forecast_within.index, forecast_within, label='Previsão VAR', color='tab:blue')

          axs[i, j].set_title(f'UF: {ufs}')
          axs[i, j].set_xlabel('Ano')
          axs[i, j].set_ylabel(ylabel)
          axs[i, j].legend()

          index += 1

    plt.tight_layout()
    plt.savefig(content_path + save_path)
    plt.show()

    metrics_df = pd.DataFrame({
        'UF': uf_valid,
        'MAE': mae_list,
        'MSE': mse_list,
        'RMSE': rmse_list,
        'MAPE': mape_list,
        'R2': r2_list,
        'Method': ['VAR' for _ in uf_valid]
    })

    return metrics_df


####### Função de plotting das métricas em gráficos salvos em uma pasta do Drive #######
def metrics_graphics(df, save_path):
  methods = ['Prophet', 'Arima', 'VAR']
  content_path = "/content/drive/MyDrive/TCC-MBA_USP_ESALQ/"

  mae_list = []
  mse_list = []
  rmse_list = []
  mape_list = []
  r2_list = []

  lista_df_methods = [group for _, group in df.groupby('Method')]
  valores_comuns = set(lista_df_methods[0]['UF'])
  for method in lista_df_methods:
    valores_comuns = valores_comuns.intersection(set(method['UF']))

  lista_df_filtrada = [df[df['UF'].isin(valores_comuns)] for df in lista_df_methods]

  for df_method in lista_df_filtrada:
    df_method = df_method.sort_values(by=['UF'])
    mae_list.append(df_method['MAE'].tolist())
    mse_list.append(df_method['MSE'].tolist())
    rmse_list.append(df_method['RMSE'].tolist())
    mape_list.append(df_method['MAPE'].tolist())
    r2_list.append(df_method['R2'].tolist())

  uf_list = list(valores_comuns)

  fig, axs = plt.subplots(5, 1, figsize=(15, 30))
  axs[0].plot(uf_list, mae_list[0], color='tab:blue', label = methods[0])
  axs[0].plot(uf_list, mae_list[1], color='black', label = methods[1])
  axs[0].plot(uf_list, mae_list[2], color='tab:red', label = methods[2])
  axs[0].set_title('MAE')
  axs[0].legend()

  axs[1].plot(uf_list, mse_list[0], color='tab:blue', label = methods[0])
  axs[1].plot(uf_list, mse_list[1], color='black', label = methods[1])
  axs[1].plot(uf_list, mse_list[2], color='tab:red', label = methods[2])
  axs[1].set_title('MSE')
  axs[1].legend()

  axs[2].plot(uf_list, rmse_list[0], color='tab:blue', label = methods[0])
  axs[2].plot(uf_list, rmse_list[1], color='black', label = methods[1])
  axs[2].plot(uf_list, rmse_list[2], color='tab:red', label = methods[2])
  axs[2].set_title('RMSE')
  axs[2].legend()

  axs[3].plot(uf_list, mape_list[0], color='tab:blue', label = methods[0])
  axs[3].plot(uf_list, mape_list[1], color='black', label = methods[1])
  axs[3].plot(uf_list, mape_list[2], color='tab:red', label = methods[2])
  axs[3].set_title('MAPE')
  axs[3].legend()

  axs[4].plot(uf_list, r2_list[0], color='tab:blue', label = methods[0])
  axs[4].plot(uf_list, r2_list[1], color='black', label = methods[1])
  axs[4].plot(uf_list, r2_list[2], color='tab:red', label = methods[2])
  axs[4].set_title('R2')
  axs[4].legend()

  plt.tight_layout()
  plt.savefig(content_path + save_path)
  plt.show()

In [ ]:
################ Todos os DataFrames Meteorológicos ################

start = time.time()

path_met_tot = '/content/drive/MyDrive/TCC-MBA_USP_ESALQ/Meteorologia/'
onlyfiles = [f for f in listdir(path_met_tot) if not isfile(join(path_met_tot, f))]
df_met_general_list = []
for dir in onlyfiles:
  df_met_info, df_met_general = read_all_met_csv_from_path(path_met_tot + dir)
  df_met_general = data_and_hour_to_datetime(df_met_general)
  df_met_general = take_mean_weather(df_met_general)
  df_met_general_list.append(df_met_general)

df_met_general = pl.concat(df_met_general_list)

end = time.time()

weather_export_time = end - start

In [ ]:
################ Todos os DataFrames dos Produtos ################

start = time.time()

################ Path e Dataframe referentes à Cana de Açúcar ################
path_cana = 'Produtos/Cana/'
df_cana = read_all_csv_from_path(path_default + path_cana)
df_cana = take_avg_df(df_cana)
df_cana = df_cana.join(df_met_general, on=['UF', 'Ano'], how='left')
df_cana = year_to_date(df_cana)

################ Path e Dataframe referentes à Milho ################
path_milho = 'Produtos/Milho/'
df_milho = read_all_csv_from_path(path_default + path_milho)
df_milho  = take_avg_df(df_milho)
df_milho = df_milho.join(df_met_general, on=['UF', 'Ano'], how='left')
df_milho = year_to_date(df_milho)

################ Path e Dataframe referentes à Soja ################
path_soja = 'Produtos/Soja/'
df_soja = read_all_csv_from_path(path_default + path_soja)
df_soja = take_avg_df(df_soja)
df_soja = df_soja.join(df_met_general, on=['UF', 'Ano'], how='left')
df_soja = year_to_date(df_soja)

end = time.time()

product_export_time = end - start

In [ ]:
################ Path e Dataframe referentes à Cana de Açúcar sem join com os dados meteorológicos ################
path_cana = 'Produtos/Cana/'
df_cana_solo = read_all_csv_from_path(path_default + path_cana)
df_cana_solo = take_avg_df(df_cana_solo)
df_cana_solo = year_to_date(df_cana_solo)

################ Path e Dataframe referentes à Milho sem join com os dados meteorológicos  ################
path_milho = 'Produtos/Milho/'
df_milho_solo = read_all_csv_from_path(path_default + path_milho)
df_milho_solo  = take_avg_df(df_milho_solo)
df_milho_solo = year_to_date(df_milho_solo)

################ Path e Dataframe referentes à Soja sem join com os dados meteorológicos  ################
path_soja = 'Produtos/Soja/'
df_soja_solo = read_all_csv_from_path(path_default + path_soja)
df_soja_solo = take_avg_df(df_soja_solo)
df_soja_solo = year_to_date(df_soja_solo)

# Análise da Área Colhida dos Produtos

### Usando o Prophet

In [ ]:
df_cana_area_solo = df_cana_solo.filter(pl.col("Nomes de medida") == 'Área Colhida')
df_milho_area_solo = df_milho_solo.filter(pl.col("Nomes de medida") == 'Área Colhida')
df_soja_area_solo = df_soja_solo.filter(pl.col("Nomes de medida") == 'Área Colhida')

In [ ]:
df_metricas_prophet_cana_area_solo = prophet_forecast(df_cana_area_solo, 'Área Colhida', "plots_prophet/forecast_cana_colhida.png")
df_metricas_prophet_milho_area_solo = prophet_forecast(df_milho_area_solo, 'Área Colhida', "plots_prophet/forecast_milho_colhida.png")
df_metricas_prophet_soja_area_solo = prophet_forecast(df_soja_area_solo, 'Área Colhida', "plots_prophet/forecast_milho_colhida.png")

### Usando AutoARIMA

In [ ]:
df_metricas_arima_cana_area_solo = autoarima_forecast(df_cana_area_solo, 'Área Colhida', "plots_arima/forecast_cana_colhida.png")
df_metricas_arima_milho_area_solo = autoarima_forecast(df_milho_area_solo, 'Área Colhida', "plots_arima/forecast_milho_colhida.png")
df_metricas_arima_soja_area_solo = autoarima_forecast(df_soja_area_solo, 'Área Colhida', "plots_arima/forecast_milho_colhida.png")

### Usando o VAR

In [ ]:
df_cana_area = df_cana.filter(pl.col("Nomes de medida") == 'Área Colhida')
df_milho_area = df_milho.filter(pl.col("Nomes de medida") == 'Área Colhida')
df_soja_area = df_soja.filter(pl.col("Nomes de medida") == 'Área Colhida')

In [ ]:
df_metricas_var_cana_area = var_forecast(df_cana_area, 'Área Colhida', "plots_var/forecast_cana_colhida.png")
df_metricas_var_milho_area = var_forecast(df_milho_area, 'Área Colhida', "plots_var/forecast_milho_colhida.png")
df_metricas_var_soja_area = var_forecast(df_soja_area, 'Área Colhida', "plots_var/forecast_milho_colhida.png")

## Análise de Métricas

In [ ]:
df_metricas_total_cana_area_solo = pd.concat([df_metricas_prophet_cana_area_solo, df_metricas_arima_cana_area_solo, df_metricas_var_cana_area])
df_metricas_total_milho_area_solo = pd.concat([df_metricas_prophet_milho_area_solo, df_metricas_arima_milho_area_solo, df_metricas_var_milho_area])
df_metricas_total_soja_area_solo = pd.concat([df_metricas_prophet_soja_area_solo, df_metricas_arima_soja_area_solo, df_metricas_var_soja_area])

df_metricas_total_cana_area_solo.to_excel('/content/drive/MyDrive/TCC-MBA_USP_ESALQ/metrics/metrics_cana_area_solo.xlsx')
df_metricas_total_milho_area_solo.to_excel('/content/drive/MyDrive/TCC-MBA_USP_ESALQ/metrics/metrics_milho_area_solo.xlsx')
df_metricas_total_soja_area_solo.to_excel('/content/drive/MyDrive/TCC-MBA_USP_ESALQ/metrics/metrics_soja_area_solo.xlsx')

metrics_graphics(df_metricas_total_cana_area_solo, 'plots_metrics/metrics_cana_area_solo.png')
metrics_graphics(df_metricas_total_milho_area_solo, 'plots_metrics/metrics_milho_area_solo.png')
metrics_graphics(df_metricas_total_soja_area_solo, 'plots_metrics/metrics_soja_area_solo.png')

# Análise de Média de Produtividade dos Produtos

In [ ]:
df_cana_medprod_solo = df_cana_solo.filter(pl.col("Nomes de medida") == 'Média de Produtividade (Área Colhida)')
df_milho_medprod_solo = df_milho_solo.filter(pl.col("Nomes de medida") == 'Média de Produtividade (Área Colhida)')
df_soja_medprod_solo = df_soja_solo.filter(pl.col("Nomes de medida") == 'Média de Produtividade (Área Colhida)')

### Usando o Prophet

In [ ]:
df_metricas_prophet_cana_medprod_solo = prophet_forecast(df_cana_medprod_solo, 'Média de Produtividade', "plots_prophet/forecast_cana_media.png")
df_metricas_prophet_milho_medprod_solo = prophet_forecast(df_milho_medprod_solo, 'Média de Produtividade', "plots_prophet/forecast_milho_media.png")
df_metricas_prophet_soja_medprod_solo = prophet_forecast(df_soja_medprod_solo, 'Média de Produtividade', "plots_prophet/forecast_soja_media.png")

### Usando o AutoARIMA

In [ ]:
df_metricas_arima_cana_medprod_solo = autoarima_forecast(df_cana_medprod_solo, 'Média de Produtividade', "plots_arima/forecast_cana_media.png")
df_metricas_arima_milho_medprod_solo = autoarima_forecast(df_milho_medprod_solo, 'Média de Produtividade', "plots_arima/forecast_milho_media.png")
df_metricas_arima_soja_medprod_solo = autoarima_forecast(df_soja_medprod_solo, 'Média de Produtividade', "plots_arima/forecast_soja_media.png")

## Usando o VAR

In [ ]:
df_cana_medprod = df_cana.filter(pl.col("Nomes de medida") == 'Média de Produtividade (Área Colhida)')
df_milho_medprod = df_milho.filter(pl.col("Nomes de medida") == 'Média de Produtividade (Área Colhida)')
df_soja_medprod = df_soja.filter(pl.col("Nomes de medida") == 'Média de Produtividade (Área Colhida)')

In [ ]:
df_metricas_var_cana_medprod = var_forecast(df_cana_medprod, 'Média de Produtividade', "plots_var/forecast_cana_media.png")
df_metricas_var_milho_medprod = var_forecast(df_milho_medprod, 'Média de Produtividade', "plots_var/forecast_milho_media.png")
df_metricas_var_soja_medprod = var_forecast(df_soja_medprod, 'Média de Produtividade', "plots_var/forecast_milho_media.png")

## Análise de Métricas

In [ ]:
df_metricas_total_cana_medprod_solo = pd.concat([df_metricas_prophet_cana_medprod_solo, df_metricas_arima_cana_medprod_solo, df_metricas_var_cana_medprod])
df_metricas_total_milho_medprod_solo = pd.concat([df_metricas_prophet_milho_medprod_solo, df_metricas_arima_milho_medprod_solo, df_metricas_var_milho_medprod])
df_metricas_total_soja_medprod_solo = pd.concat([df_metricas_prophet_soja_medprod_solo, df_metricas_arima_soja_medprod_solo, df_metricas_var_soja_medprod])

df_metricas_total_cana_medprod_solo.to_excel('/content/drive/MyDrive/TCC-MBA_USP_ESALQ/metrics/metrics_cana_medprod_solo.xlsx')
df_metricas_total_milho_medprod_solo.to_excel('/content/drive/MyDrive/TCC-MBA_USP_ESALQ/metrics/metrics_milho_medprod_solo.xlsx')
df_metricas_total_soja_medprod_solo.to_excel('/content/drive/MyDrive/TCC-MBA_USP_ESALQ/metrics/metrics_soja_medprod_solo.xlsx')

metrics_graphics(df_metricas_total_cana_medprod_solo, 'plots_metrics/metrics_cana_medprod_solo.png')
metrics_graphics(df_metricas_total_milho_medprod_solo, 'plots_metrics/metrics_milho_medprod_solo.png')
metrics_graphics(df_metricas_total_soja_medprod_solo, 'plots_metrics/metrics_soja_medprod_solo.png')

# Análise de Área Plantada dos Produtos

In [ ]:
df_cana_plant_solo = df_cana_solo.filter(pl.col("Nomes de medida") == 'Área Plantada')
df_milho_plant_solo = df_milho_solo.filter(pl.col("Nomes de medida") == 'Área Plantada')
df_soja_plant_solo = df_soja_solo.filter(pl.col("Nomes de medida") == 'Área Plantada')

### Usando o Prophet

In [ ]:
df_metricas_prophet_cana_plant_solo = prophet_forecast(df_cana_plant_solo, 'Área Plantada', "plots_prophet/forecast_cana_plantada.png")
df_metricas_prophet_milho_plant_solo = prophet_forecast(df_milho_plant_solo, 'Área Plantada', "plots_prophet/forecast_milho_plantada.png")
df_metricas_prophet_soja_plant_solo = prophet_forecast(df_soja_plant_solo, 'Área Plantada', "plots_prophet/forecast_soja_plantada.png")

### Usando o AutoARIMA

In [ ]:
df_metricas_arima_cana_plant_solo = autoarima_forecast(df_cana_plant_solo, 'Área Plantada', "plots_arima/forecast_cana_plantada.png")
df_metricas_arima_milho_plant_solo = autoarima_forecast(df_milho_plant_solo, 'Área Plantada', "plots_arima/forecast_milho_plantada.png")
df_metricas_arima_soja_plant_solo = autoarima_forecast(df_soja_plant_solo, 'Área Plantada', "plots_arima/forecast_soja_plantada.png")

## Usando o VAR

In [ ]:
df_cana_plant = df_cana.filter(pl.col("Nomes de medida") == 'Área Plantada')
df_milho_plant = df_milho.filter(pl.col("Nomes de medida") == 'Área Plantada')
df_soja_plant = df_soja.filter(pl.col("Nomes de medida") == 'Área Plantada')

In [ ]:
df_metricas_var_cana_plant = var_forecast(df_cana_plant, 'Área Plantada', "plots_var/forecast_cana_plantada.png")
df_metricas_var_milho_plant = var_forecast(df_milho_plant, 'Área Plantada', "plots_var/forecast_milho_plantada.png")
df_metricas_var_soja_plant = var_forecast(df_soja_plant, 'Área Plantada', "plots_var/forecast_milho_plantada.png")

## Análise de Métricas

In [ ]:
df_metricas_total_cana_plant_solo = pd.concat([df_metricas_prophet_cana_plant_solo, df_metricas_arima_cana_plant_solo, df_metricas_var_cana_plant])
df_metricas_total_milho_plant_solo = pd.concat([df_metricas_prophet_milho_plant_solo, df_metricas_arima_milho_plant_solo, df_metricas_var_milho_plant])
df_metricas_total_soja_plant_solo = pd.concat([df_metricas_prophet_soja_plant_solo, df_metricas_arima_soja_plant_solo, df_metricas_var_soja_plant])

df_metricas_total_cana_plant_solo.to_excel('/content/drive/MyDrive/TCC-MBA_USP_ESALQ/metrics/metrics_cana_plant_solo.xlsx')
df_metricas_total_milho_plant_solo.to_excel('/content/drive/MyDrive/TCC-MBA_USP_ESALQ/metrics/metrics_milho_plant_solo.xlsx')
df_metricas_total_soja_plant_solo.to_excel('/content/drive/MyDrive/TCC-MBA_USP_ESALQ/metrics/metrics_soja_plant_solo.xlsx')

metrics_graphics(df_metricas_total_cana_plant_solo, 'plots_metrics/metrics_cana_plant_solo.png')
metrics_graphics(df_metricas_total_milho_plant_solo, 'plots_metrics/metrics_milho_plant_solo.png')
metrics_graphics(df_metricas_total_soja_plant_solo, 'plots_metrics/metrics_soja_plant_solo.png')

# Análise de Quantidade Produzida pelos Produtos

In [ ]:
df_cana_qnt_solo = df_cana_solo.filter(pl.col("Nomes de medida") == 'Qtde Produzida')
df_milho_qnt_solo = df_milho_solo.filter(pl.col("Nomes de medida") == 'Qtde Produzida')
df_soja_qnt_solo = df_soja_solo.filter(pl.col("Nomes de medida") == 'Qtde Produzida')

### Usando o Prophet

In [ ]:
df_metricas_prophet_cana_qnt_solo = prophet_forecast(df_cana_qnt_solo, 'Qtde Produzida', "plots_prophet/forecast_cana_produzida.png")
df_metricas_prophet_milho_qnt_solo = prophet_forecast(df_milho_qnt_solo, 'Qtde Produzida', "plots_prophet/forecast_milho_produzida.png")
df_metricas_prophet_soja_qnt_solo = prophet_forecast(df_soja_qnt_solo, 'Qtde Produzida', "plots_prophet/forecast_soja_produzida.png")

### Usando o AutoARIMA

In [ ]:
df_metricas_arima_cana_qnt_solo = autoarima_forecast(df_cana_qnt_solo, 'Qtde Produzida', "plots_arima/forecast_cana_produzida.png")
df_metricas_arima_milho_qnt_solo = autoarima_forecast(df_milho_qnt_solo, 'Qtde Produzida', "plots_arima/forecast_milho_produzida.png")
df_metricas_arima_soja_qnt_solo = autoarima_forecast(df_soja_qnt_solo, 'Qtde Produzida', "plots_arima/forecast_soja_produzida.png")

## Usando o VAR

In [ ]:
df_cana_qnt = df_cana.filter(pl.col("Nomes de medida") == 'Qtde Produzida')
df_milho_qnt = df_milho.filter(pl.col("Nomes de medida") == 'Qtde Produzida')
df_soja_qnt = df_soja.filter(pl.col("Nomes de medida") == 'Qtde Produzida')

In [ ]:
df_metricas_var_cana_qnt = var_forecast(df_cana_qnt, 'Qtde Produzida', "plots_var/forecast_cana_produzida.png")
df_metricas_var_milho_qnt = var_forecast(df_milho_qnt, 'Qtde Produzida', "plots_var/forecast_milho_produzida.png")
df_metricas_var_soja_qnt = var_forecast(df_soja_qnt, 'Qtde Produzida', "plots_var/forecast_milho_produzida.png")

## Análise de Métricas

In [ ]:
df_metricas_total_cana_qnt_solo = pd.concat([df_metricas_prophet_cana_qnt_solo, df_metricas_arima_cana_qnt_solo, df_metricas_var_cana_qnt])
df_metricas_total_milho_qnt_solo = pd.concat([df_metricas_prophet_milho_qnt_solo, df_metricas_arima_milho_qnt_solo, df_metricas_var_milho_qnt])
df_metricas_total_soja_qnt_solo = pd.concat([df_metricas_prophet_soja_qnt_solo, df_metricas_arima_soja_qnt_solo, df_metricas_var_soja_qnt])

df_metricas_total_cana_qnt_solo.to_excel('/content/drive/MyDrive/TCC-MBA_USP_ESALQ/metrics/metrics_cana_qnt_solo.xlsx')
df_metricas_total_milho_qnt_solo.to_excel('/content/drive/MyDrive/TCC-MBA_USP_ESALQ/metrics/metrics_milho_qnt_solo.xlsx')
df_metricas_total_soja_qnt_solo.to_excel('/content/drive/MyDrive/TCC-MBA_USP_ESALQ/metrics/metrics_soja_qnt_solo.xlsx')

metrics_graphics(df_metricas_total_cana_qnt_solo, 'plots_metrics/metrics_cana_qnt_solo.png')
metrics_graphics(df_metricas_total_milho_qnt_solo, 'plots_metrics/metrics_milho_qnt_solo.png')
metrics_graphics(df_metricas_total_soja_qnt_solo, 'plots_metrics/metrics_soja_qnt_solo.png')